In [21]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [22]:
df = pd.read_csv('short.csv')

In [23]:
df['action'].value_counts()

action
find_doctor         4000
find_medicalshop    4000
find_hospital       4000
find_lab            4000
book_appointment    4000
my_appointments     4000
my_records          4000
Name: count, dtype: int64

In [24]:
label_encoder = LabelEncoder()
df['action'] = label_encoder.fit_transform(df['action'])

In [25]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [26]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['query'])
total_words = len(tokenizer.word_index) + 1

In [27]:
train_sequences = tokenizer.texts_to_sequences(train_data['query'])
test_sequences = tokenizer.texts_to_sequences(test_data['query'])

In [28]:
train_padded_sequences = pad_sequences(train_sequences)
test_padded_sequences = pad_sequences(test_sequences, maxlen=train_padded_sequences.shape[1])

In [47]:
train_padded_sequences.shape[1]

16

In [29]:
df['action'].unique()

array([1, 4, 2, 3, 0, 5, 6])

In [30]:
train_labels = tf.keras.utils.to_categorical(train_data['action'], num_classes=len(df['action'].unique()))
test_labels = tf.keras.utils.to_categorical(test_data['action'], num_classes=len(df['action'].unique()))

In [31]:
model = Sequential()
model.add(Embedding(total_words, 64, input_length=train_padded_sequences.shape[1]))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(len(set(df['action'])), activation='softmax'))

In [32]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(train_padded_sequences, train_labels, epochs=5, verbose=1, validation_data=(test_padded_sequences, test_labels))

Epoch 1/5
700/700 [==============================] - 5s 5ms/step - loss: 1.0433 - accuracy: 0.5850 - val_loss: 0.8019 - val_accuracy: 0.6466
Epoch 2/5
700/700 [==============================] - 3s 4ms/step - loss: 0.7726 - accuracy: 0.6586 - val_loss: 0.7808 - val_accuracy: 0.6498
Epoch 3/5
700/700 [==============================] - 3s 4ms/step - loss: 0.7554 - accuracy: 0.6612 - val_loss: 0.7704 - val_accuracy: 0.6452
Epoch 4/5
700/700 [==============================] - 3s 4ms/step - loss: 0.7468 - accuracy: 0.6660 - val_loss: 0.7677 - val_accuracy: 0.6375
Epoch 5/5
700/700 [==============================] - 3s 4ms/step - loss: 0.7412 - accuracy: 0.6649 - val_loss: 0.7635 - val_accuracy: 0.6555


In [40]:
new_queries = ["Need this neighborhood drugstore agra.",""]

In [41]:
new_sequences = tokenizer.texts_to_sequences(new_queries)
new_padded_sequences = pad_sequences(new_sequences, maxlen=train_padded_sequences.shape[1])

In [42]:
predictions = model.predict(new_padded_sequences)
predicted_classes = [label_encoder.classes_[tf.argmax(prediction).numpy()] for prediction in predictions]

1/1 [==============================] - 0s 19ms/step


In [43]:
predicted_classes

['find_medicalshop', 'find_doctor']

In [38]:
model.evaluate(test_padded_sequences, test_labels)

175/175 [==============================] - 0s 2ms/step - loss: 0.7635 - accuracy: 0.6555


[0.7635149955749512, 0.6555356979370117]

In [39]:
model.save('action.keras')

In [45]:
import pickle

with open('action_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [46]:
with open('action_label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)